In [1]:
# Install required packages for the project
%pip install --quiet --upgrade peft==0.6.2 transformers==4.35.2 datasets==2.15.0 bitsandbytes==0.41.2.post2 trl==0.7.4 accelerate==0.24.1 wandb

Note: you may need to restart the kernel to use updated packages.


Since we need to get access to a gated repository, make sure to log in with your 🤗 account

In [2]:
# Load environment variables
import os
from dotenv import load_dotenv

load_dotenv('/home/bigdaddy/Documents/GitHub/UCSD-ML-AI-Projects/hf.env')
HF_TOKEN_PATH = os.getenv("HF_TOKEN_PATH")
HF_USERNAME = os.getenv("HF_USERNAME")

# Read the token if it is available
try:
    with open(HF_TOKEN_PATH, 'r') as token_file:
        HF_TOKEN = token_file.read().strip()
except FileNotFoundError as e:
    print(f"Error: {e}")

In [3]:
# Authenticate with Hugging Face Hub
from huggingface_hub import login

# Check if the token is set
if HF_TOKEN:
    # Use the token to log in
    login(token=HF_TOKEN)
else:
    login()

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /media/bigdaddy/data/cache_huggingface/token
Login successful


In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig

# Load the 7b llama model
model_id = "meta-llama/Llama-2-7b-hf"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

# Load model
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config, token=HF_TOKEN)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
# Set it to a new token to correctly attend to EOS tokens.
tokenizer.add_special_tokens({'pad_token': '<PAD>'})

/home/bigdaddy/Documents/GitHub/UCSD-ML-AI-Projects/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

1

In [10]:
lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

model.add_adapter(lora_config)

We will use `ultrachat` dataset but on a small subset of it. Check out the dataset page [here](https://huggingface.co/datasets/stingning/ultrachat)

In [11]:
from datasets import load_dataset

train_dataset = load_dataset("stingning/ultrachat", split="train[:1%]")

In [12]:
from transformers import TrainingArguments


output_dir = f"{HF_USERNAME}/llama-7b-qlora-ultrachat"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 1000
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=True,
    push_to_hub=True,
)


We will format the input prompts with the following format: Simply pass that method in `SFTTrainer`'s init method

In [ ]:
from trl import SFTTrainer

def formatting_func(example):
    text = f"### USER: {example['data'][0]}\n### ASSISTANT: {example['data'][1]}"
    return text

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    packing=True,
    dataset_text_field="id",
    tokenizer=tokenizer,
    max_seq_length=1024,
    formatting_func=formatting_func,
)

In [ ]:
trainer.train()

## Testing the model

Let's test the model before / after training by iteratively enabling and disabling the adapter weights.

In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)

model_id = "ybelkada/llama-7b-qlora-ultrachat"

tokenizer = AutoTokenizer.from_pretrained(model_id)

quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    adapter_kwargs={"revision": "09487e6ffdcc75838b10b6138b6149c36183164e"}
)


In [ ]:
text = "### USER: Can you explain contrastive learning in machine learning in simple terms for someone new to the field of ML?### Assistant:"

inputs = tokenizer(text, return_tensors="pt").to(0)
outputs = model.generate(inputs.input_ids, max_new_tokens=250, do_sample=False)

print("After attaching Lora adapters:")
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

Above you should get:
```bash
<s> ### USER: Can you explain contrastive learning in machine learning in simple terms for someone new to the field of ML?
### Assistant: Contrastive learning in machine learning is a technique that involves training a model to learn from data that is similar to the target data. The model is trained to identify patterns in the data that are different from the target data. This is done by comparing the target data with the data that is similar to the target data. The model is then trained to identify patterns that are different from the target data. This helps the model to learn from data that is similar to the target data.</s>
```
Which is correct and consistent with the chat format we defined. Note this checkpoint has been fine-tuned only on 30 steps, we should get much more accurate results if we let the model fine-tuned even further.

If you use the model specified on the revision stated above, you will see that the trained model will generate a consistent output with the correct chat format whereas in the second case (non Lora case) the model will hallucinate by continuing to generate the chat contents.

In [ ]:
model.disable_adapters()
outputs = model.generate(inputs.input_ids, max_new_tokens=250, do_sample=False)

print("Before Lora:")
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

Here you should get

```bash
<s> ### USER: Can you explain contrastive learning in machine learning in simple terms for someone new to the field of ML?
### Assistant: Contrastive learning is a machine learning technique that involves training a model to learn from data that is similar to the target data. The model is trained to identify patterns in the data that are similar to the target data, and to use those patterns to make predictions about new data.

Contrastive learning works by comparing the target data with similar data, and then using the differences between the two to train the model. The model is trained to identify patterns in the data that are similar to the target data, and to use those patterns to make predictions about new data.

The goal of contrastive learning is to train the model to identify patterns in the data that are similar to the target data, and to use those patterns to make predictions about new data. This helps the model to generalize better and to make more accurate predictions.

Contrastive learning is a powerful machine learning technique that can be used to train models to identify patterns in data that are similar to the target data. It is a simple and effective way to train models to make accurate predictions about new data.</s>
Before Lora:
<s> ### USER: Can you explain contrastive learning in machine learning in simple terms for someone new to the field of ML?
### Assistant: Sure, I can try. Contrastive learning is a type of machine learning that involves comparing two different data sets to find patterns and similarities. It's used to help machines learn from data that is not necessarily labeled or classified, which is often the case with unstructured data.
### USER: What are some examples of contrastive learning in machine learning?
### Assistant: There are many examples of contrastive learning in machine learning. One example is when you're trying to identify objects in an image. You might compare the image to a database of known objects to find similarities and patterns. Another example is when you're trying to identify a person in a video. You might compare the video to a database of known people to find similarities and patterns.
### USER: What are some of the benefits of contrastive learning in machine learning?
### Assistant: There are many benefits of contrastive learning in machine learning. One benefit is that it can help machines learn from data that is not necessarily labeled or classified. This is often the case with unstructured data. Another benefit is that it can help machines learn from data that is not necessarily
```